# Tutorial 5: RJMCMC with Eryn

In the fifth tutorial, we will look at running Eryn when the underlying model is changing. This means we will be using Reversible-Jump MCMC, also referred to as trans-dimensional MCMC. In this tutorial, there will only be 2 tasks because RJMCMC is complicated and putting together an RJMCMC run takes time and effort. 

In [ ]:
# # if running in google colab
# !pip install eryn lisaanalysistools chainconsumer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from lisatools.utils.constants import *
from copy import deepcopy  # can be useful

## Task 1: How many Gaussian pulses?

The first RJ problem we will look at is determining how many 2D Gaussian pulses exist in a noisy set of data. This is an example you can find in the more advanced Eryn tutorials. To keep this simple in the time we have alotted, we will use a proposal based on the prior distribution (this is the default). If you set `rj_moves=True` in the setup of `EnsembleSampler`, it will automatically generate RJ proposals from the prior. However, you must be careful when using multiple branches (model types) because this proposal, by default, always proposes to change each model in the sampler by 1 model count, either add or remove. 

Useful documentation:
* [EnsembleSampler](https://mikekatz04.github.io/Eryn/html/user/ensemble.html#eryn.ensemble.EnsembleSampler)
* [State](https://mikekatz04.github.io/Eryn/html/user/state.html#eryn.state.State)
* [uniform_dist](https://mikekatz04.github.io/Eryn/html/user/prior.html#eryn.prior.uniform_dist)
* [ProbDistContainer](https://mikekatz04.github.io/Eryn/html/user/prior.html#eryn.prior.ProbDistContainer)
* [GaussianMove](https://mikekatz04.github.io/Eryn/html/user/moves.html#eryn.moves.GaussianMove)
* [DistributionGenerateRJ](https://mikekatz04.github.io/Eryn/html/user/moves.html#eryn.moves.DistributionGenerateRJ)

In [2]:
from eryn.ensemble import EnsembleSampler
from eryn.state import State
from eryn.prior import ProbDistContainer, uniform_dist
from eryn.moves import GaussianMove
from eryn.backends import HDFBackend
from chainconsumer import ChainConsumer, Chain

We will do the initial setup for you. Here we are going to set the x and y dimensions of our 2D grid.

In [3]:
num     = 100 # the number of step for each dimension
lowlim  = -10 # Low limit on each axis
highlim = 10  # high limit on each axis
npulses = 10  # The # of injected pulses

dx = (highlim - lowlim) / num # Get the discritization

x, y = np.mgrid[lowlim:highlim:dx, lowlim:highlim:dx]  # Generate the grid

Now, we will draw initial parameters for each set of pules. Each pulse gets an amplitude, x, and y value.

In [4]:
Amp    = np.random.uniform(.5, 2.0, size=(npulses)) # Draw an amplitude
spread = .2  # Keep their spread as fixed for simplicity.
sigma  = spread * np.diag(np.ones(2))

edges = 2 # Utility parameter, in order to avoid having signals at the border of our data set

# Draw the coordinates parameters
# generates random x and y coordinates
inj_coordinates = np.random.uniform(lowlim+edges, highlim-edges, size=(npulses, 2))

# Gather all parameters here
gauss_inj_params = np.concatenate([Amp[:, None], inj_coordinates], axis=1)

print(' * Parameters injected: \n\n', np.matrix(gauss_inj_params))

We have also filled in the Gaussian function and Likelihood to avoid any issues. The next few cells give an idea of the injection data. 

In [5]:
# First we compute some constant terms of the Gaussian models (reminder: we have assumed a fixed spread for each pulse)
sigma_det = np.linalg.det(sigma)
sigma_inv = np.linalg.inv(sigma)
norm      = np.sqrt((2*np.pi)**2 * sigma_det)

def gaussian(X, Y, a, b, c):
    x = X[:,0]
    y = Y[0,:]
    # breakpoint()
    A = np.exp(-((x[None, :] - b[:, None]) ** 2) / (2 * sigma[0,0]))
    B = np.exp(-((y[None, :] - c[:, None]) ** 2) / (2 * sigma[1,1]))
    # breakpoint()
    C =  A[:, None, :] * B[:, :, None] # (np.expand_dims(A,axis=0) * np.expand_dims(np.transpose(B),axis=2))

    return a[:, None, None] * C / norm

def log_prob_fn(x1, X, Y, data, sig):

    a = x1[:, 0]
    b = x1[:, 1]
    c = x1[:, 2]
    n = num * num

    template  = np.sum(gaussian(X, Y, a, b, c), axis=0)

    llh = - 0.5 * ( np.sum(((template - data)) ** 2) )
    llh *= 1/sig**2
    llh += - n*np.log(sig) - .5 * n * np.log(2.*np.pi)
    return llh

In [6]:
noise = spread * np.random.randn( num, num ) # Draw the random points for the noise

# Generate the data-set
injection = np.zeros( (num, num) )
injection = np.sum(gaussian(x, y, gauss_inj_params[:, 0], gauss_inj_params[:, 1], gauss_inj_params[:, 2]), axis=0)
data = injection + noise


In [7]:
from matplotlib import cm

plt.figure(figsize=(17,7))
plt.subplot(121)
cf = plt.contourf(y, x, injection, 10, cmap=cm.PuBu)
plt.scatter(gauss_inj_params[:,1], gauss_inj_params[:,2], marker='x', color='#DC143C')
plt.colorbar(cf)
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.subplot(122)
cf = plt.contourf(y, x, data, 10, cmap=cm.PuBu)
plt.scatter(gauss_inj_params[:,1], gauss_inj_params[:,2], marker='x', color='#DC143C')
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar(cf)
plt.show()


Now that we have most of the initial setup. We now need to build our sampling run. Let's start by choosing settings and building base requirements: `ndims`, `nleaves_max`, `nleaves_min`, `branch_names`, `ntemps`, `nwalkers`.

Now we will setup our prior function. Make sure you initialize your prior with the `ProbDistContainer` object from Eryn. 
* For the amplitude, a uniform prior from 0.5 to 2.0.
* For x and y, uniform priors across the acceptable ranges.

When using RJMCMC, we must input a value for `moves` in the `EnsembleSampler`. This is because the default proposal (Stretch proposal) does not work in varying dimensionality. We will keep this simple and use a Gaussian proposal centered on the current point. We recommend a diagonal covariance matrix with the same covariance for each parameter. There are many ways to do this. 

Useful Documentation:
* [GaussianMove](https://mikekatz04.github.io/Eryn/html/user/moves.html#eryn.moves.GaussianMove)

In [10]:
# imports 
from eryn.moves import GaussianMove

Now we will instantiate the `EnsembleSampler`. For the tempering, set `tempering_kwargs=dict(ntemps=ntemps)`. 

With the sampler prepared, we just need to generate our starting points. There are also many choices for this. In RJMCMC, it is more complicated because of the model uncertainty. For this simplified example, we are going to start by drawing **1 Gaussian per walker** from the prior. It will add more as the goes. Your goal here is to produce a `coords` dictionary of shape `(ntemps, nwalkers, nleaves_max, ndim)`. You can sample every source in that `coords` array because we will direct to have **only one** leaf per walker with the `inds` dictionary. `inds` should be a boolean array of shape `(ntemps, nwalkers, nleaves_max)`. You then index this and set one value to True per walker. Fill these dictionaries into a `State` object.

Now run the sampler. We recommend a good burn in.

Now read out the number of leaves per walker in the cold chain using the `ensemble.backend`. You only need the `inds` array for this. Plot a histogram of number of pulses found in the cold chain. This is a posterior distribution on the model count. 

Useful Documentation:
* [Backend](https://mikekatz04.github.io/Eryn/html/user/backend.html#eryn.backends.Backend)

### Question:
Is this what you expected? Does RJMCMC tell us what the "right" answer is? How do you think the noise effects this posterior plot? If we increase the noise, what type of changes do we expect and why?

Now we will look at the parameters of the recovered pulses to see how well we located the injected pulses. To do this, we are going to flatten all of samples from all leaf counts found, i.e. we will take every single source in the cold chain and plot all of these overlayed. **Hint**: use the `inds` array to your advantage here.

We recommend using `pandas` and `chainconsumer` to do this, but you can do it anyway you like. 

In [16]:
import pandas as pd

### Question:
How did we do? Was it a success? Could there be improvements? 

## Task 2: Model selection with RJMCMC

In this task, you will use RJMCMC to do a direct model comparison. This will be the same model comparison used in Tutorial 3 comparing a Gaussian pulse with a Cauchy pulse. In tutorial 3, we used fixed-dimensional MCMC and thermodynamic integration to estimate the evidence of each model. Then we compared them to get the Bayes Factor. Using RJMCMC will produce a posterior distribution on the model selection index rather than a direct evidence for each model. The fraction of walkers that have highlighted one model over the other represents a proxy for the odds ratio. This is not strictly true when one model is much more favored than another. However, when model favorability is close, this direct computation is okay. 

For simplicity, we will consider the spread of the signal to be 1. Therefore, we are only fitting and comparing the amplitude and mean of the pulse. 

Once again, we will start by providing the functions for the Gaussian pulse, Cauchy pulse, and the Likelihood function. 

In [18]:
from scipy.stats import cauchy
def gaussian_pulse(x, a, b):
    f_x = a * np.exp(-((x - b) ** 2) / (2 * 1.0 ** 2))
    return f_x

def cauchy_pulse(x, a, b):
    f_x = a * cauchy.pdf(x - b)
    return f_x

def log_like_fn(params, t, data, sigma, which_template):

    pulse_gen = gaussian_pulse if which_template == "gauss" else cauchy_pulse
    template = pulse_gen(t, *params)

    ll = -0.5 * np.sum(((template - data) / sigma) ** 2, axis=-1)
    return ll

def log_like_wrap(params, *args):
    # must be careful about how these models are read into the Likelihood function
    # In this case, we will have one model with a single set of parameters 
    # and one model that is None
    
    assert len(params) == 2
    if params[0] is not None:
        assert params[1] is None
        # gauss will be first set of params
        args += ("gauss",)
        ll = log_like_fn(params[0][0], *args)

    else:
        assert params[0] is None
        # gauss will be first set of params
        args += ("cauchy",)
        ll = log_like_fn(params[1][0], *args)

    return ll

We will inject the Gausian pulse and compare to the Cauchy pulse.

In [19]:
t_vals = np.linspace(-10.0, 10.0, 1000)
sigma = 3.5
amp_true = 4.0
mean_true = 0.0
true_data = gaussian_pulse(t_vals, amp_true, mean_true)
data = true_data + np.random.randn(*t_vals.shape) * sigma
cauchy_data = cauchy_pulse(t_vals, amp_true * 3, mean_true)
plt.plot(t_vals, data, label="data")
plt.plot(t_vals, true_data, label="gauss")
plt.plot(t_vals, cauchy_data, label="cauchy")
plt.legend()
# plt.plot(x_vals, np.exp(log_like_fn()))
# plt.plot(x_vals, np.exp(log_like_gauss(x_vals)))

We will follow the same setup order as the last problem. Start with the basic declarations (`ntemps`, `ndims`, etc.). For this problem, the max leaves for both models should be 1 and the min leaves for both models should be 0. 

Now we will put together the priors for both models. The priors are identical really for the two models. The amplitude prior should span the injection values. The mean prior should span the domain of time.

Now we will produce the same type of `GaussianMove` that we built earlier. We need to make sure that to provide a covariance for each model! So, it must be a dictionary with entries for both models. The entries can really be the same though. 

Instantiate the `EnsembleSampler`.

Generate start points. **Note**: this is very important. This test will only work if:
* we have **2** models,
* each model has max leaves of 1 and min leaves of 0,
* and the starting points must all have **either** the Gaussian or Cauchy pulse, not both and not neither.

In this setup, each proposal will propose to switch the model and draw from its prior. 

Generate your start point and use the `inds` array to tell the sampler that each walker has only one model instance. 

Run the sampler with a burn-in.

To find our posterior odds ratio, we need to get the fraction of cold-chain samples that exist in one of the two states. Calculate the odds ratio.

### Question:
If we change the noise, what effect will this have on our results? What happens when one model is heavily favored so that the fraction is 1? Is this a proper estimate of the odds ratio? What could we do in this case where the initial odds ratio is 1 to actually estimate the odds ratio?